<a href="https://colab.research.google.com/github/00hamza00/CE888-Data-Science-and-Decision-Making/blob/master/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Lab 5. Recommender Systems**

In [0]:
import pandas as pd
import numpy as np

In [0]:
col_names = ['JokeCount'] + [f'Joke{num + 1}' for num in range(100)]
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv',header=None, names=col_names)

In [0]:
df = df.drop(columns=['JokeCount'])

In [0]:
arr = df.values

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, validation_indices = replace(arr, 0.1)

In [0]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [9]:
sgd()

26.174728697671405
23.230410425191423
20.724293614111428
17.861758485554724
17.27238052670391


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = [f'Joke{num + 1}' for num in range(100)]

In [11]:
comparison_data

,Joke1,Joke2,Joke3,Joke4,Joke5,Joke6,Joke7,Joke8,Joke9,Joke10,Joke11,Joke12,Joke13,Joke14,Joke15,Joke16,Joke17,Joke18,Joke19,Joke20,Joke21,Joke22,Joke23,Joke24,Joke25,Joke26,Joke27,Joke28,Joke29,Joke30,Joke31,Joke32,Joke33,Joke34,Joke35,Joke36,Joke37,Joke38,Joke39,Joke40,...,Joke61,Joke62,Joke63,Joke64,Joke65,Joke66,Joke67,Joke68,Joke69,Joke70,Joke71,Joke72,Joke73,Joke74,Joke75,Joke76,Joke77,Joke78,Joke79,Joke80,Joke81,Joke82,Joke83,Joke84,Joke85,Joke86,Joke87,Joke88,Joke89,Joke90,Joke91,Joke92,Joke93,Joke94,Joke95,Joke96,Joke97,Joke98,Joke99,Joke100
0,"(-7.82, -3.8337921379568924)","(8.79, -4.5368166558561045)","(-9.66, -5.55819669069625)","(-8.16, -9.387199314077074)","(-7.52, -2.861230102316446)","(-8.5, -1.9591631318252671)","(-9.85, -5.045663839067049)","(4.17, -4.271602430106377)","(-8.98, -8.464637778378307)","(-4.76, -3.6649278625895376)","(-8.5, -1.7813600706309602)","(-6.75, -2.0401794222174603)","(-7.18, -6.786211443448577)","(8.45, -1.9997185646560705)","(99.0, -6.652298923031264)","(99.0, -10.172413750085877)","(-7.43, -4.731433972904381)","(-9.81, -5.365118900770029)","(-9.85, -3.9736937616033647)","(-9.85, -6.283315645783777)","(-9.37, -0.07298966276822713)","(1.5, -4.579664532386915)","(-4.37, -6.161687057024382)","(-9.81, -10.289608870117533)","(-8.5, -4.927380335144801)","(1.12, -2.1189069020851297)","(7.82, 3.162659747001731)","(2.86, -1.925797509250411)","(9.13, 3.11711610007871)","(99.0, -7.793683570231402)","(2.14, 0.6852687716136814)","(-4.08, 3.083021702055195)","(-9.08, -9.274378550763112)","(7.82, -3.085037427461941)","(99.0, 2.5421650853019075)","(4.95, 3.4801934068692724)","(-9.17, -8.92902901114563)","(-8.4, -3.597309157754596)","(-8.4, -3.0084172730108345)","(-8.4, -3.9064465088935214)",...,"(8.59, 0.8804510042340743)","(3.59, 1.9156585047871422)","(-6.84, -5.4246011281485735)","(-9.03, -8.158528534925363)","(2.82, 0.9279390744472381)","(-1.36, 1.1653478943898659)","(-9.08, -8.695902273846947)","(99.0, 1.528508434700855)","(5.68, 1.3996570392589798)","(-4.81, -4.856375873637595)","(99.0, -6.145163658427074)","(99.0, 1.4300080508245343)","(99.0, -2.9353445253294193)","(99.0, -8.576559856164357)","(99.0, -5.646414638249715)","(99.0, 0.7543573130045361)","(99.0, -3.685574203671)","(-9.42, -1.7161175608280892)","(99.0, -4.852703802850877)","(99.0, -2.4877257261991)","(99.0, -1.1069788466453176)","(-7.72, -3.957259684419427)","(99.0, -0.12555224874231258)","(99.0, -4.074957152224043)","(99.0, -3.769088598505168)","(99.0, -5.43627590327606)","(99.0, -0.7632555113546241)","(99.0, -0.7849416696380833)","(99.0, 3.4551955933691434)","(99.0, -4.577741129187002)","(2.82, -0.8661722945587216)","(99.0, -2.629483885438598)","(99.0, 0.5275784807934384)","(99.0, -4.002162198939926)","(99.0, -3.367366915021583)","(99.0, -2.369230019181242)","(-5.63, -1.806040114507376)","(99.0, -2.4181387095924696)","(99.0, -5.699703161754457)","(99.0, -2.479687738633951)"
1,"(4.08, 3.114570846217256)","(-0.29, 3.500369319372579)","(6.36, 3.957428892824273)","(4.37, 5.485355745235314)","(-2.38, 2.4188860406746575)","(-9.66, 2.7254715325593692)","(-0.73, 3.2245462835198517)","(-5.34, 2.4490961527893)","(8.88, 5.178177608299368)","(9.22, 3.129626538077663)","(6.75, 2.5988596652336176)","(99.0, 2.8231292350468205)","(4.42, 3.4934401599878644)","(7.43, 2.807136830897232)","(4.56, 3.2746734411865774)","(-0.97, 4.694349106009166)","(99.0, 2.7365159656755917)","(-0.68, 3.1700056539902954)","(3.3, 2.950755765391619)","(-1.21, 3.888790257473445)","(0.87, 1.9273554804018393)","(8.64, 3.811796897609122)","(8.35, 4.292479623548188)","(9.17, 5.7057096481721405)","(0.05, 3.88093715700064)","(7.57, 2.811968896143645)","(4.71, 0.33732351879963324)","(0.87, 2.640457214110587)","(-0.39, 0.2747829609365003)","(6.99, 5.075736394810944)","(6.5, 1.4990813555507747)","(-0.92, 0.37459090619008595)","(7.14, 5.4429313543953555)","(9.03, 3.1114736494554442)","(-1.8, 0.6584272152476395)","(0.73, 0.1126644853875198)","(7.09, 5.045623227999694)","(3.4, 3.294759

In [12]:
original_data = arr
assert len(validation_indices[0]) == len(validation_indices[1])
validation_size = len(validation_indices[0])
errors = []
for i in range(validation_size):
  user_id = validation_indices[0][i]
  joke_id = validation_indices[1][i]
  prediction_rating = predict_rating(user_id, joke_id)
  real_rating = original_data[user_id][joke_id]
  error = prediction_rating - real_rating
  errors.append(error)
mse = (np.array(errors) ** 2).mean()
print('Validation MSE:', mse)

Validation MSE: 18.087336648629467


In [0]:
filled_data = np.zeros(original_data.shape)
rows, columns = original_data.shape
for user_id in range(rows):
  for joke_id in range(columns):
    if original_data[user_id][joke_id] == 99:
      filled_data[user_id][joke_id] = predict_rating(user_id, joke_id)
    else:
      filled_data[user_id][joke_id] = original_data[user_id][joke_id]

In [14]:
filled_data

array([[-7.82      ,  8.79      , -9.66      , ..., -2.41813871,
        -5.69970316, -2.47968774],
       [ 4.08      , -0.29      ,  6.36      , ...,  0.34      ,
        -4.32      ,  1.07      ],
       [ 6.30561   ,  6.61517702,  6.59234325, ...,  6.85550484,
         6.17357993,  7.76153337],
       ...,
       [-0.3747293 , -0.45905594, -0.59028141, ..., -0.18329371,
        -0.61612399, -0.17447245],
       [-1.37599046, -1.73600759, -2.31916089, ..., -0.50181335,
        -2.45281286, -0.42154349],
       [ 2.43      ,  2.67      , -3.98      , ...,  3.56540193,
         2.04847958,  4.12175741]])

In [0]:
means = np.mean(filled_data, axis=0)

In [16]:
max_joke = -9999999999
max_joke_index = -1
min_joke = 99999999999
min_joke_index = -1

for i in range(len(means)):
  if means[i] > max_joke:
    max_joke = means[i]
    max_joke_index = i
  elif means[i] < min_joke:
    min_joke = means[i]
    min_joke_index = i

print(f'The best-rated joke was joke {max_joke_index + 1}, with a mean rating of: {max_joke}')
print(f'The worst-rated joke was joke {min_joke_index + 1}, with a mean rating of: {min_joke}')

The best-rated joke was joke 50, with a mean rating of: 3.66529908474608
The worst-rated joke was joke 58, with a mean rating of: -3.7431870989393228
